In [1]:
from src.vector_store.vector_storage import VectorStorage
import pandas as pd

df = pd.read_csv('qa.csv')
vec_db = VectorStorage()

In [2]:
import time

for i, row in df.iterrows():
    if row['content_type_id'] == 2:
        vec = vec_db.vector_query_event(row['question'])
        key = vec_db.keyword_query_event(row['question'])
        hybrid = vec_db.hybrid_query_event(row['question'])
    else:
        vec = vec_db.vector_query_base(row['question'])
        key = vec_db.keyword_query_base(row['question'])
        hybrid = vec_db.hybrid_query_base(row['question'])

    found_vec = row['header'] in str(vec)
    found_key = row['header'] in str(key)
    found_hybrid = row['header'] in str(hybrid)

    df.at[i, 'found_vec'] = found_vec
    df.at[i, 'found_key'] = found_key
    df.at[i, 'found_hybrid'] = found_hybrid
    time.sleep(35)


In [3]:
df.to_csv('qa_eval.csv', index=False)

In [9]:
all_vec = df['found_vec'].sum()
all_key = df['found_key'].sum()
all_hybrid = df['found_hybrid'].sum()

all_vec_event = df[df['content_type_id'] == 2]['found_vec'].sum()
all_key_event = df[df['content_type_id'] == 2]['found_key'].sum()
all_hybrid_event = df[df['content_type_id'] == 2]['found_hybrid'].sum()

all_vec_base = df[df['content_type_id'] == 1]['found_vec'].sum()
all_key_base = df[df['content_type_id'] == 1]['found_key'].sum()
all_hybrid_base = df[df['content_type_id'] == 1]['found_hybrid'].sum()

all_vec_event_percentage = all_vec_event / df[df['content_type_id'] == 2].shape[0]
all_key_event_percentage = all_key_event / df[df['content_type_id'] == 2].shape[0]
all_hybrid_event_percentage = all_hybrid_event / df[df['content_type_id'] == 2].shape[0]

all_vec_base_percentage = all_vec_base / df[df['content_type_id'] == 1].shape[0]
all_key_base_percentage = all_key_base / df[df['content_type_id'] == 1].shape[0]
all_hybrid_base_percentage = all_hybrid_base / df[df['content_type_id'] == 1].shape[0]

eval_df = pd.DataFrame({'vector': [all_vec, all_vec_event_percentage, all_vec_base_percentage],
                        'keyword': [all_key, all_key_event_percentage, all_key_base_percentage],
                        'hybrid': [all_hybrid, all_hybrid_event_percentage, all_hybrid_base_percentage]},
                       index=['Celkem', 'EventSchema', 'BaseSchema'])

eval_df.to_csv('qa_eval_summary.csv')

display(eval_df)
